In [ ]:
# Must be included at the beginning of each new notebook. Remember to change the app name.
import findspark
findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Data_Transformation').getOrCreate()

In [ ]:
# Let's read in the data. Note that it's in the format of JSON.
dog_data_merged = spark.read.load("Datasets/Dog_registred_hamilton_new_v1_2_5.csv", format="csv", header ="true")
dog_data_merged.show()

In [ ]:
dog_data_merged.groupBy("Classification").count().show()

In [ ]:
from pyspark.sql.functions import when

dog_data_merged = dog_data_merged.withColumn("Classification", \
              when(dog_data_merged["Classification"] == "Unsafe", "1").\
              otherwise("0"))

In [ ]:
dog_data_merged.printSchema()

In [ ]:
from pyspark.sql.types import IntegerType
dog_data_merged = dog_data_merged.withColumn("Dog_Number",dog_data_merged["Dog_Number"].cast(IntegerType()))
dog_data_merged = dog_data_merged.withColumn("Age",dog_data_merged["Age"].cast(IntegerType()))
dog_data_merged = dog_data_merged.withColumn("Total_Complains",dog_data_merged["Total_Complains"].cast(IntegerType()))
dog_data_merged = dog_data_merged.withColumn("Classification",dog_data_merged["Classification"].cast(IntegerType()))

In [ ]:
dog_data_merged.printSchema()

In [ ]:
dog_data_merged.groupBy("Classification").count().show()

In [ ]:
dog_dataPanda_merged = dog_data_merged.toPandas()

In [ ]:
import plotly.express as px

fig = px.box(dog_dataPanda_merged, y="Age")
fig.show()

In [ ]:
dog_data_merged.select("Age").distinct().orderBy("Age").show(n=999999, truncate=True)

In [ ]:
dog_data_merged.filter("Age >= 28").count()

In [ ]:
dog_data_merged.count()

In [ ]:
dog_data_merged.select("Total_Complains").distinct().orderBy("Total_Complains").show(n=999999, truncate=True)

In [ ]:
fig_complains = px.box(dog_dataPanda_merged, y="Total_Complains")
fig_complains.show()

In [ ]:
quantilesAge = dog_data_merged.approxQuantile("Age", [0.25, 0.5, 0.75], 0)

In [ ]:
quantilesAge

In [ ]:
quantilesComplains = dog_data_merged.approxQuantile("Total_Complains", [0.25, 0.5, 0.75], 0)

In [ ]:
quantilesComplains

In [ ]:
iqrAge = (10-2)
iqrAge

In [ ]:
iqrComplains = (2-0)
iqrComplains

In [ ]:
lowerBoundaryAge = 2 - 1.5 * iqrAge
lowerBoundaryAge

In [ ]:
upperBoundaryAge = 10 + 1.5 * iqrAge
upperBoundaryAge

In [ ]:
lowerBoundaryComplains = 0 - 1.5 * iqrComplains
lowerBoundaryComplains

In [ ]:
upperBoundaryComplains = 2 + 1.5 * iqrComplains
upperBoundaryComplains

In [ ]:
from pyspark.sql.functions import when
dog_data_merged = dog_data_merged.withColumn("Age", \
              when(dog_data_merged["Age"] >22, -1).otherwise(dog_data_merged["Age"]))

In [ ]:
from pyspark.sql.functions import when
dog_data_merged = dog_data_merged.withColumn("Total_Complains", \
              when(dog_data_merged["Total_Complains"] >22, -1).otherwise(dog_data_merged["Total_Complains"]))

In [ ]:
dog_data_merged.filter(dog_data_merged.Age == -1).count()

In [ ]:
dog_data_merged.filter(dog_data_merged.Total_Complains == -1).count()

In [ ]:
dog_data_merged = dog_data_merged.filter(dog_data_merged.Age > -1)
dog_data_merged.filter(dog_data_merged.Age == -1).count()

In [ ]:
dog_data_merged = dog_data_merged.filter(dog_data_merged.Total_Complains > -1)
dog_data_merged.filter(dog_data_merged.Total_Complains == -1).count()

In [ ]:
dog_data_merged.repartition(1).write.format("com.databricks.spark.csv").option("header", "true")\
.save("Dog_registred_hamilton_new_v1_2_6.csv")